In [1]:
import torch 
import torch.nn.functional as F

label = 5
label = F.one_hot(torch.tensor(label), num_classes=6)

c:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
label

tensor([0, 0, 0, 0, 0, 1])

In [3]:
from utils import get_dataloader
train_dataloader, valid_dataloader, test_dataloader = get_dataloader('data/data.csv')

c:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [1]:
from utils import get_dataset 
train_dataset, valid_dataset, test_dataset = get_dataset('data/data.csv')

c:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
len(train_dataset[0])

2

In [2]:
import pandas as pd
data = pd.read_csv('data/data.csv')
# concatenate song name and song lyric
data['lyric'] = data['song name'] + " " + data['song lyric']
# convert genre to numeric
mapping = {'COUNTRY': 0,
            'HIPHOP': 1,
            'INDIE': 2,
            'JAZZ': 3,
            'POP': 4,
            'ROCK': 5}
data['genre'] = data['category'].map(mapping)

In [3]:
data.tail()

,index,song name,song lyric,category,lyric,genre
8995,8995,Cocaine,"If you wanna hang out, you've gotta take her o...",ROCK,"Cocaine If you wanna hang out, you've gotta ta...",5
8996,8996,All Along the Watchtower,"""There must be some kind of way out of here"". ...",ROCK,"All Along the Watchtower ""There must be some k...",5
8997,8997,Carry on Wayward Son,Carry on my wayward son. There'll be peace whe...,ROCK,Carry on Wayward Son Carry on my wayward son. ...,5
8998,8998,Simple Man,Mama told me when I was young. Come sit beside...,ROCK,Simple Man Mama told me when I was young. Come...,5
8999,8999,The Prisoner - 2015 Remaster,We want information. Information. Information....,ROCK,The Prisoner - 2015 Remaster We want informati...,5


In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['lyric'], data['genre'], test_size=0.056, random_state=43)

In [5]:
len(x_train)

8496

In [5]:
y_train[5000]

3

In [6]:
from datasets import GenreDataset
train_dataset = GenreDataset(x_train, y_train)

c:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
train_dataset[0][0]['token_type_ids']

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])